# Import Dependencies

In [56]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st

# Import Data

In [57]:
income_data = pd.read_csv("./raw_data/income_Distribution_OECD_013021.csv")
income_data.head()

C:\Users\berry\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,LOCATION,Country,MEASURE,Measure,AGE,Age group,DEFINITION,Definition,METHODO,Methodology,...,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,AUS,Australia,GINI,"Gini (disposable income, post taxes and transf...",TOT,Total population,CURRENT,Current definition,METH2012,New income definition since 2012,...,2012,0_TO_1,0-1 scale,0,Units,NaN,NaN,0.326,NaN,NaN
1,AUS,Australia,GINI,"Gini (disposable income, post taxes and transf...",TOT,Total population,CURRENT,Current definition,METH2012,New income definition since 2012,...,2014,0_TO_1,0-1 scale,0,Units,NaN,NaN,0.337,NaN,NaN
2,AUS,Australia,GINI,"Gini (disposable income, post taxes and transf...",TOT,Total population,CURRENT,Current definition,METH2012,New income definition since 2012,...,2016,0_TO_1,0-1 scale,0,Units,NaN,NaN,0.330,NaN,NaN
3,AUS,Australia,GINI,"Gini (disposable income, post taxes and transf...",TOT,Total population,CURRENT,Current definition,METH2012,New income definition since 2012,...,2018,0_TO_1,0-1 scale,0,Units,NaN,NaN,0.325,NaN,NaN
4,AUS,Australia,GINI,"Gini (disposable income, post taxes and transf...",TOT,Total population,CURRENT,Current definition,METH2011,Income definition until 2011,...,2004,0_TO_1,0-1 scale,0,Units,NaN,NaN,0.315,NaN,NaN


In [58]:
union_df = pd.read_csv("./raw_data/uniondata.csv")
union_df.head()

,Unnamed: 0,Country,Source,Series,Year,Unit,Value
0,54,Hungary,Survey data,Trade union density,2004,Percentage,16.8
1,55,Hungary,Survey data,Trade union density,2009,Percentage,11.9
2,56,Hungary,Survey data,Trade union density,2015,Percentage,9.0
3,94,Poland,Administrative data,Trade union density,1999,Percentage,29.1
4,95,Poland,Administrative data,Trade union density,2000,Percentage,24.7


In [59]:
gini_cpi_df = pd.read_csv("raw_data/gini_cpi.csv")
gini_cpi_df.head()

,Unnamed: 0,Country,MEASURE,Age group,Year,Unit Code,Unit,Value
0,0,Australia,GINI,Total population,2012,0_TO_1,0-1 scale,0.326
1,1,Australia,GINI,Total population,2014,0_TO_1,0-1 scale,0.337
2,2,Australia,GINI,Total population,2016,0_TO_1,0-1 scale,0.330
3,3,Australia,GINI,Total population,2018,0_TO_1,0-1 scale,0.325
4,6,Australia,GINI,Total population,2010,0_TO_1,0-1 scale,0.334


### Merge and transform data

In [60]:
df = pd.merge(union_df, gini_cpi_df, on=["Country", "Year"], suffixes=("_union", "_gini_cpi"))
df.head()

,Unnamed: 0_union,Country,Source,Series,Year,Unit_union,Value_union,Unnamed: 0_gini_cpi,MEASURE,Age group,Unit Code,Unit_gini_cpi,Value_gini_cpi
0,55,Hungary,Survey data,Trade union density,2009,Percentage,11.9,7947,GINI,Total population,0_TO_1,0-1 scale,0.241000
1,55,Hungary,Survey data,Trade union density,2009,Percentage,11.9,7959,GINI,Total population,0_TO_1,0-1 scale,0.272000
2,55,Hungary,Survey data,Trade union density,2009,Percentage,11.9,7964,CPI2010,Total population,IDX,Index,95.369304
3,55,Hungary,Survey data,Trade union density,2009,Percentage,11.9,7976,CPI2010,Total population,IDX,Index,95.369304
4,55,Hungary,Survey data,Trade union density,2009,Percentage,11.9,29555,GINI,Working age population: 18-65,0_TO_1,0-1 scale,0.245000


### Remove unrelated columns

In [61]:
df_09_18 = df[["Country", "Year", "Value_union", "MEASURE", "Age group", "Value_gini_cpi"]]
df_09_18.head()

,Country,Year,Value_union,MEASURE,Age group,Value_gini_cpi
0,Hungary,2009,11.9,GINI,Total population,0.241000
1,Hungary,2009,11.9,GINI,Total population,0.272000
2,Hungary,2009,11.9,CPI2010,Total population,95.369304
3,Hungary,2009,11.9,CPI2010,Total population,95.369304
4,Hungary,2009,11.9,GINI,Working age population: 18-65,0.245000


### Working age population: 18~65

In [62]:
working_age = df_09_18[df_09_18["Age group"] == "Working age population: 18-65"]

gini_working = working_age[working_age["MEASURE"] == "GINI"]
gini_working = gini_working.drop_duplicates(subset=["Country","Year"])

cpi_working = working_age[working_age["MEASURE"] != "GINI"]
cpi_working = cpi_working.drop_duplicates(subset=["Country","Year"])

# Merge two dfs: gigi_working, cpi_working
# Drop unrelated columns
data_working = pd.merge(gini_working, cpi_working, on=["Country", "Year"])\
                        .drop(columns=["MEASURE_x", "Age group_x", "Value_union_y", "MEASURE_y", "Age group_y"])
# Rename columns 
data_working = data_working.rename(columns = {
                                                "Value_union_x": "Union %",
                                                "Value_gini_cpi_x": "Gini",
                                                "Value_gini_cpi_y": "CPI"
                                            })
data_working

,Country,Year,Union %,Gini,CPI
0,Hungary,2009,11.9,0.245,95.369304
1,Hungary,2015,9.0,0.291,111.384411
2,Poland,2009,16.3,0.305,97.484226
3,Poland,2010,18.3,0.307,100.000000
4,Poland,2011,17.0,0.304,104.239401
...,...,...,...,...,...
269,United States,2013,10.8,0.392,106.833852
270,United States,2014,10.7,0.389,108.566940
271,United States,2015,10.6,0.384,108.695723
272,United States,2016,10.3,0.385,110.067028


In [63]:
data_working.head()

,Country,Year,Union %,Gini,CPI
0,Hungary,2009,11.9,0.245,95.369304
1,Hungary,2015,9.0,0.291,111.384411
2,Poland,2009,16.3,0.305,97.484226
3,Poland,2010,18.3,0.307,100.000000
4,Poland,2011,17.0,0.304,104.239401


In [64]:
#rename columns
data_working=data_working.rename(columns={"Union %":"Union Density","Gini":"Gini Coefficient"})
data_working

,Country,Year,Union Density,Gini Coefficient,CPI
0,Hungary,2009,11.9,0.245,95.369304
1,Hungary,2015,9.0,0.291,111.384411
2,Poland,2009,16.3,0.305,97.484226
3,Poland,2010,18.3,0.307,100.000000
4,Poland,2011,17.0,0.304,104.239401
...,...,...,...,...,...
269,United States,2013,10.8,0.392,106.833852
270,United States,2014,10.7,0.389,108.566940
271,United States,2015,10.6,0.384,108.695723
272,United States,2016,10.3,0.385,110.067028


In [65]:
west = ['United Kingdom', 'Spain', 'Austria', 'France', 'Germany', 'Ireland', 'Switzerland', 'Italy', 'Luxembourg', 
        'Greece', 'Israel', 'Netherlands', 'Belgium', 'Australia', 'New Zealand', 'Portugal', 'Turkey', 'Canada']
east =  ['Estonia', 'Czech Republic', 'Latvia', 'Lithuania', 'Hungary', 'Slovak Republic', 'Slovenia', 'Poland'] 
scandinavian = ['Norway', 'Finland' ,'Iceland', 'Sweden', 'Denmark']
us = ['United States']
asia = ['Korea', 'Japan']
latin = ['Mexico', 'Chile']

In [66]:
regions = {
    "West": west,
    "East": east,
    "Scandinavia": scandinavian,
    "US": us,
    "Asia": asia,
    "Latin": latin
}
working = data_working
working["Region"] = ""
for index, row in working.iterrows():
    for region in regions:
        if row.Country in regions[region]: 
            working.loc[index, "Region"] = region
            break
            
working.head()

,Country,Year,Union Density,Gini Coefficient,CPI,Region
0,Hungary,2009,11.9,0.245,95.369304,East
1,Hungary,2015,9.0,0.291,111.384411,East
2,Poland,2009,16.3,0.305,97.484226,East
3,Poland,2010,18.3,0.307,100.000000,East
4,Poland,2011,17.0,0.304,104.239401,East


In [67]:
west_df = pd.DataFrame({"Country": west})
east_df = pd.DataFrame({"Country": east})
scand_df = pd.DataFrame({"Country": scandinavian})
us_df = pd.DataFrame({"Country": us})
asia_df = pd.DataFrame({"Country": asia})
latin_df = pd.DataFrame({"Country": latin})

In [68]:
west_filter = pd.merge(data_working, west_df, on=["Country"])
east_filter = pd.merge(data_working, east_df, on=["Country"])
scand_filter = pd.merge(data_working, scand_df, on=["Country"])
us_filter = pd.merge(data_working, us_df, on=["Country"])
asia_filter = pd.merge(data_working, asia_df, on=["Country"])
latin_filter = pd.merge(data_working, latin_df, on=["Country"])

df = {
    "west": {"df": west_filter, "x":0, "y":0, "color": "c"},
    "east": {"df": east_filter, "x":0, "y":1, "color": "m"},
    "scand": {"df":scand_filter, "x":1, "y":0, "color": "y"},
    "us": {"df":us_filter, "x":1, "y":1, "color": "b"},
    "asia": {"df":asia_filter, "x":2, "y":0, "color": "g"},
    "latin": {"df":latin_filter, "x":2, "y":1, "color": "r"}
}

In [69]:
data_working
data_working["Color"] = ""
data_working["Group"] = ""
data_working.head()

,Country,Year,Union Density,Gini Coefficient,CPI,Region,Color,Group
0,Hungary,2009,11.9,0.245,95.369304,East,,
1,Hungary,2015,9.0,0.291,111.384411,East,,
2,Poland,2009,16.3,0.305,97.484226,East,,
3,Poland,2010,18.3,0.307,100.000000,East,,
4,Poland,2011,17.0,0.304,104.239401,East,,


In [70]:
for index, row in data_working.iterrows():
    if row["Country"] in west: 
        data_working.loc[index,"Group"] = 10
        data_working.loc[index,"Color"] = "c"
    if row["Country"] in east: 
        data_working.loc[index,"Group"] = 50 
        data_working.loc[index,"Color"] = "m"
    if row["Country"] in scandinavian: 
        data_working.loc[index,"Group"] = 95
        data_working.loc[index,"Color"] = "y"
    if row["Country"] in us: 
        data_working.loc[index,"Group"] = 130 
        data_working.loc[index,"Color"] = "b"
    if row["Country"] in asia: 
        data_working.loc[index,"Group"] = 160
        data_working.loc[index,"Color"] = "g"
    if row["Country"] in latin: 
        data_working.loc[index,"Group"] = 190
        data_working.loc[index,"Color"] = "r"
data_working.head()
# data_working.plot.scatter(x="Union %", y="Gini", c="Group", colormap="viridis")

,Country,Year,Union Density,Gini Coefficient,CPI,Region,Color,Group
0,Hungary,2009,11.9,0.245,95.369304,East,m,50
1,Hungary,2015,9.0,0.291,111.384411,East,m,50
2,Poland,2009,16.3,0.305,97.484226,East,m,50
3,Poland,2010,18.3,0.307,100.000000,East,m,50
4,Poland,2011,17.0,0.304,104.239401,East,m,50


In [71]:
data_working.drop(['CPI'],axis=1)

,Country,Year,Union Density,Gini Coefficient,Region,Color,Group
0,Hungary,2009,11.9,0.245,East,m,50
1,Hungary,2015,9.0,0.291,East,m,50
2,Poland,2009,16.3,0.305,East,m,50
3,Poland,2010,18.3,0.307,East,m,50
4,Poland,2011,17.0,0.304,East,m,50
...,...,...,...,...,...,...,...
269,United States,2013,10.8,0.392,US,b,130
270,United States,2014,10.7,0.389,US,b,130
271,United States,2015,10.6,0.384,US,b,130
272,United States,2016,10.3,0.385,US,b,130


## Export to .csv

In [72]:
data_working.to_csv('output/data_working.csv')